# Naive (Basic) RAG
Uses simplistic retrieval methods, such as keyword matching, semantic similarity using Vector DB.

In [ ]:
logged_retriever = retriever | RunnableLambda(log_chunks) | format_docs

rag_chain = (
    {"context": logged_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Simple RAG with Memory
Integrating a storage component, allowing the model to retain information from previous interactions. This addition was crucial for enabling contextual awareness across multiple queries, particularly in continuous conversations, often achieved through prompt caching

In [ ]:
# Add history using memory
session_histories = {}

def get_session_history(session_id: str):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

# Extract question and history to pass to retriever and prompt
def input_mapper(input):
    return {
        "context": logged_retriever.invoke(input["question"]),
        "question": input["question"],
        "history": input["history"]
    }

# Chain
rag_chain = (
    RunnableLambda(input_mapper)
    | CHAT_HISTORY_PROMPT
    | llm
    | StrOutputParser()
)

rag_with_history = RunnableWithMessageHistory(
    runnable=rag_chain,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    output_messages_key="output"
)

# Branched RAG
Designed for improved efficiency, Branched RAG allows the system to evaluate the user query and intelligently select the most relevant specific data source(s) to query, rather than indiscriminately querying all available sources. This targeted approach streamlines retrieval and reduces computational overhead.

# HyDE (Hypothetical Document Embeddings)
This technique significantly improved retrieval relevance. Instead of directly retrieving documents, HyDE first creates an embedded representation of what an ideal document might look like, given the query. This hypothetical document then guides the actual retrieval process, leading to more precise and higher-quality results by addressing query-document asymmetry.